<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moviepy pyannote-audio

In [ ]:
import numpy as np
from pyannote.audio import Pipeline, Audio
from pyannote.core import Segment
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
video_path = "./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.mp4"
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
import json, re

In [ ]:
model = whisper.load_model("small")

In [ ]:
result = model.transcribe(audio_path, verbose=True, fp16=False, language="ja")

In [ ]:
segments = result['segments']
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
for seg in segments:
  count = len(prog.findall(seg['text']))
  if count > 0: print(seg['text'])
  _seg = {'start': seg['start'], 'end': seg['end'], 'text': seg['text'], 'count': count}
  INTERVIEW.append(_seg)

In [ ]:
segments = result['segments']
embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device='cpu')
embeddings = np.zeros(shape=(len(segments), 192))

duration = video.duration
embeddings = []


for i, segment in enumerate(segments):
    audio = Audio()
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(audio_path, clip)
    print(waveform[0])
    _enbed = embedding_model(waveform[None])
    #print(_enbed[0])
    embeddings.append(_enbed[0])

embeddings = np.nan_to_num(embeddings)


In [ ]:
from sklearn.cluster import AgglomerativeClustering
speaker_count = 2

In [ ]:
len(embeddings)
embeddings.shape

(96, 192)

In [ ]:
clustering = AgglomerativeClustering(speaker_count).fit(embeddings)

In [ ]:
labeled_segments = []
for label, segment in zip(clustering.labels_, result["segments"]):
    labeled_segments.append((label, segment["start"], segment["text"]))

interivew = []
print(INTERVIEW)
for s, (speaker, _, text) in enumerate(labeled_segments):
    stime = INTERVIEW[s]["start"]
    interivew.append(f"話者{speaker + 1} :  {stime} 〜 : 「{text}」")

[{'start': 0.0, 'end': 10.0, 'text': 'どうもこんにちは', 'count': 0}, {'start': 10.0, 'end': 12.0, 'text': 'ありがとうございます', 'count': 0}, {'start': 12.0, 'end': 19.0, 'text': 'ごめん 記者の時間いただきましてどうもありがとうございます', 'count': 0}, {'start': 20.0, 'end': 22.0, 'text': 'おはよう', 'count': 0}, {'start': 24.0, 'end': 29.0, 'text': 'では早速質問をさせていただきたいと思います', 'count': 0}, {'start': 29.0, 'end': 31.0, 'text': 'ちょっとだけお願いします', 'count': 1}, {'start': 31.0, 'end': 49.0, 'text': 'では早速なんですけれども', 'count': 0}, {'start': 49.0, 'end': 55.0, 'text': 'まずフリーをどうでしたでしょうか', 'count': 1}, {'start': 56.0, 'end': 62.0, 'text': '今ちょっと回線がフリーズして聞こえなかったんですけど', 'count': 0}, {'start': 62.0, 'end': 67.0, 'text': 'フリーを導入されたのはいつ頃でしたでしょうか', 'count': 2}, {'start': 67.0, 'end': 79.0, 'text': '今から6年前で2018年の4月ですね', 'count': 0}, {'start': 79.0, 'end': 83.0, 'text': 'すごい そんな前からだったんですね', 'count': 0}, {'start': 83.0, 'end': 85.0, 'text': 'はい', 'count': 0}, {'start': 85.0, 'end': 91.0, 'text': 'その当時他の例えばクラウド会計のものとかって', 'count': 0}, {'start': 91.0, 'end': 93

In [ ]:
for speek in interivew:
  print(speek)